In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import wandb
from scripts.models import *
from scripts.utils import custom_metric, Wandb_plot, csv_logger, animate_function
import pandas as pd
import matplotlib.animation as animation
import ast
import networkx as nx
from matplotlib.animation import FuncAnimation, PillowWriter

In [10]:
m = Model(1, [1,64,64,1], [None, tf.keras.activations.tanh,tf.keras.activations.tanh,tf.keras.activations.tanh,None])
m

[layer : layer_0, neurons: 1, layer : layer_1, neurons: 64, layer : layer_2, neurons: 64, layer : layer_3, neurons: 1]

In [4]:
x_train = np.arange(-10, 10, 0.01)
y_train = np.sin(x_train)
y_mean = np.mean(y_train)

In [5]:
# Define learning rate schedule and optimizer
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=1000,
    decay_rate=0.9)

loss_metric = custom_metric(name = "loss")
accuracy_metric = custom_metric(name = "accuracy")

metrics = {'loss': loss_metric, 'accuracy': accuracy_metric}

optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=lr_schedule)
m.compile(optimizer, metrics, y_mean)
w_callback = Wandb_plot()
logs = csv_logger(name = "test1.csv")
#wandb.config.update({"model": m})

In [6]:
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
x_train = tf.reshape(x_train, (-1,1))
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
y_train = tf.reshape(y_train, (-1,1))

In [7]:
m.fit(x_train, y_train, epochs=1, batch_size=10, callbacks=[logs])

2024-08-24 22:14:36.878400: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


200/200 [==============================] - 14s 48ms/step - loss: 198.0573 - accuracy: -15.6423


In [ ]:
m.predict(x_train).numpy()

In [ ]:
animate_function(logs= 'logs/test1.csv', name = "test1.gif")

In [ ]:
m.save("model1")
